# Question 12-5
Below is my code for Q12-5! The question is the following: 

Perform a Monte Carlo simulation of the finite $6 \times 6$ system, using $10^4$ equilibration steps and $2\times 10^5$ data collection steps, sampled every 1000 steps, where $B=0,J=-1$ and for a temperature range of $1\leq K_BT\leq 10$. Plot the results of the mean net magnetization, mean energy, and heat capacity as a function of temperature. 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import sympy as sy

In [ ]:
def arrayPlot(array,vmin=0,vmax=None):
    """A python version of the ArrayPlot[] function in Mathematica"""
    fig, ax = plt.subplots()
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(array,cmap='Greys',vmin=vmin,vmax=vmax)
    plt.show()
def runMCmovie(kT,J,B,MCstepFunction, ax,config):
    """a python version of the runMCmovie function"""
    newConfig = config.copy()
    trajectory = []
    for _ in range(1500):
        newConfig = MCstepFunction(kT,J,B,newConfig)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        arrayplot = [ax.imshow(newConfig,cmap='Greys',
                               vmin=-1,vmax=1, animated=True)]
        # print(arrayplot)
        trajectory.append(arrayplot)
    return trajectory #a list of arrayplot outputs

In [ ]:
def energyIsing2D(config,J,B):
    """
    Function for calculating the energy of an Ising Model.
    Requires a config (the Ising Model in question), a value of J (interaction parameter)
    and a value for B (external field parameter)
    """
    energyB,energyJ = 0,0 # Initializes local variables
    for row in range(len(config)): # Loop over rows
        for col in range(len(config)): # Simultaneously loop over columns
            energyB += B*config[row,col]
            if row>0: # Up
                energyJ += J*config[row-1,col]*config[row,col] # Interior
            else:
                energyJ += J*config[len(config)-1,col]*config[row,col] # Periodic boundary conditions
            if row<len(config)-1: # Down
                energyJ += J*config[row+1,col]*config[row,col] # Interior
            else:
                energyJ += J*config[0,col]*config[row,col] # Periodic boundary conditions
            if col>0: # Left
                energyJ += J*config[row,col-1]*config[row,col] # Interior
            else:
                energyJ += J*config[row,len(config)-1]*config[row,col] # Periodic boundary conditions
            if col<len(config)-1: # right
                energyJ += J*config[row,col+1]*config[row,col] # Interior
            else:
                energyJ += J*config[row,0]*config[row,col] # Periodic boundary conditions
    return energyB + energyJ/2 # Removes double counting

def MCstepPBC(kT,J,B,config):
    """MC step for Periodic Ising Model"""
    newConfig = config.copy()
    row,col = np.random.randint(0,len(config),2)
    Estart = config[row,col]*B
    if row>0: # Up
        Estart += J*config[row-1,col]*config[row,col] # Interior
    else:
        Estart += J*config[len(config)-1,col]*config[row,col] # Periodic boundary conditions
    if row<len(config)-1: # Down
        Estart += J*config[row+1,col]*config[row,col] # Interior
    else:
        Estart += J*config[0,col]*config[row,col] # Periodic boundary conditions
    if col>0: # Left
        Estart += J*config[row,col-1]*config[row,col] # Interior
    else:
        Estart += J*config[row,len(config)-1]*config[row,col] # Periodic boundary conditions
    if col<len(config)-1: # right
        Estart += J*config[row,col+1]*config[row,col] # Interior
    else:
        Estart += J*config[row,0]*config[row,col] # Periodic boundary conditions
    Eend = -Estart

    if Eend < Estart: #did energy decrease?
      newConfig[row,col] *= -1 # Then flip the spin
    else:
      if np.all(np.random.random() <= np.exp(-(Eend-Estart)/kT)): # If a random number is less than the probability
        newConfig[row,col] *= -1 # Then flip the spin

    return newConfig # Return the potentially altered array

In [ ]:
def netMagnetization(config):
    """Python version of the second netMagnetization[] function Schrier writes
    automatically includes summation over all levels of the array"""
    return np.sum(config)
def netMagnetizationPerSpin(config):
  """Python version of net magnetization function"""
  return netMagnetization(config)/len(config)**2
def mean_net_magnetization(config):
    return np.abs(netMagnetizationPerSpin(config))
def energyPerSpin(config,J,B,energyFunction):
    """"calculate the energy per spin based on a
    particular ising model configuation "config",
    an interaction parameter "J"
    an external field parameter "B"
    and a function for calculating the energy"""
    return energyFunction(config,J,B)/len(config)**2
def mean_energy(config, J, B):
    return energyPerSpin(config, J, B, energyIsing2D)

def heat_capacity(config, J, B, kT):
    return energyPerSpin(config, J, B, energyIsing2D)**2 / (kT**2)

In [ ]:
def runMC(kT, J, B, nEquil, nDataCol, sampleInterval, config, MCstepFunction, energyFunction):
    """function to run the Ising model MC simulation and collectsamples"""
    results = {'temperature': [], 'mean_net_magnetization': [], 'mean_energy': [], 'heat_capacity': []}
    Esamples = []
    Msamples = []
    for Temp in kT:
        print(f"Running simulation for temperature kT={Temp}")
        newConfig = config.copy()
        for _ in range(nEquil):
            newConfig = MCstepFunction(Temp, J, B, newConfig)
        for i in range(nDataCol):
            newConfig = MCstepFunction(Temp, J, B, newConfig)
            if i % sampleInterval == 0:
                Esamples.append(energyFunction(newConfig,J,B))
                Msamples.append(abs(netMagnetizationPerSpin(newConfig)))
        Esamples = np.array(Esamples)
        Msamples = np.array(Msamples)
        results['temperature'].append(Temp)
        results['mean_net_magnetization'].append(Msamples.mean())
        results['mean_energy'].append(Esamples.mean()/len(newConfig)**2)
        results['heat_capacity'].append(Esamples.var()/(len(newConfig)**2*Temp**2))
        Esamples = Esamples.tolist()
        Msamples = Msamples.tolist()
    return results 

In [ ]:
temperature_range = np.linspace(1, 10, 20)  # Adjust the temperature range as needed
equilibration_steps = 10**4
data_collection_steps = 2 * 10**5
sample_interval = 1000
lattice_size = 6
J=-1 
B=0
checkerboardConfig = np.array([[(-1)**(row+col) for row in range(lattice_size)] for col in range(lattice_size)])

simulation_results = runMC(temperature_range, J, B, equilibration_steps, data_collection_steps, sample_interval, checkerboardConfig, MCstepPBC, energyIsing2D)

In [ ]:
print(simulation_results)

In [ ]:
# Plot the results using Seaborn
import seaborn as sns
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.lineplot(x=simulation_results['temperature'], y=simulation_results['mean_net_magnetization'], marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
sns.lineplot(x=simulation_results['temperature'], y=simulation_results['mean_energy'], marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
sns.lineplot(x=simulation_results['temperature'], y=simulation_results['heat_capacity'], marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

plt.tight_layout()
plt.show()

### Results 
The results here makes sense for what we would expect! We do expect the mean net magnetitization to go down as temperature increases, and we also expect for the mean energy to also increase opposite to the mean net magnetiziation. We also would expect a phase transition to occur around 2.6 which happens. Thus, we have successfully modeled this system!

# Question 13-7

Below is my code for Q13-7! The question is the following: 

Consider a $10\times 10$ periodic 2D Ising model wehre the interactions within the rows alternate between ferromagnetic ($J=-1$) and antiferromagnetic ($J=+1$) nearest-neighbor interactions, and the interactions between rows are always ferromagnetic ($J=-1$). First, modify the energy evaluation and Monte Carlo updating functions to implement this new model. Then calculate the net magnetization and magnetic susceptibility as a function of temperature, $1\leq K_BT/|J| \leq 4$. Describe how the results for the interdigitated system differ from the entirely ferromagnetic and antiferromagnetic systems. Speculate on why the differencei n phase transition (as a function of temperature) is useful for measuring the magnetic fields in hard drives. 

### Importing Packages
The main packages I use within this code are numpy and matplotlib. 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import sympy as sy

### Functions for Animation Movies
I copied the the functions from the previous code from the Jupyter Notebooks. One thing to note though, runMCMovie now has an input ```config``` so that we can look an different starting configurations. 

In [ ]:
def arrayPlot(array,vmin=0,vmax=None):
    """A python version of the ArrayPlot[] function in Mathematica"""
    fig, ax = plt.subplots()
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(array,cmap='Greys',vmin=vmin,vmax=vmax)
    plt.show()
def runMCmovie(kT,J,B,MCstepFunction, ax,config):
    """a python version of the runMCmovie function"""
    newConfig = config.copy()
    trajectory = []
    for _ in range(1500):
        newConfig = MCstepFunction(kT,J,B,newConfig)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        arrayplot = [ax.imshow(newConfig,cmap='Greys',
                               vmin=-1,vmax=1, animated=True)]
        # print(arrayplot)
        trajectory.append(arrayplot)
    return trajectory #a list of arrayplot outputs

### RunMC Function

We copied the same ```netMagnetization```, ```netMagnetizationPerSpin```, and ```runMC``` from the previous Ising Model Jupyter Notebook. One thing to note in the ```runMC``` portion is that I am now calculating the value ```Esamples.var()*len(config)**2/(kT**2), # MS per spin``` which is the calculation for Magnet Suspectibility per Spin. 

In [ ]:
def netMagnetization(config):
    """Python version of the second netMagnetization[] function Schrier writes
    automatically includes summation over all levels of the array"""
    return np.sum(config)
def netMagnetizationPerSpin(config):
  """Python version of net magnetization function"""
  return netMagnetization(config)/len(config)**2
def runMC(kT, J, B, nEquil, nDataCol, sampleInterval, config, MCstepFunction, energyFunction):
    """function to run the Ising model MC simulation and collectsamples"""
    Esamples = []
    Msamples = []
    newConfig = config.copy()
    for i in range(nEquil):
        newConfig = MCstepFunction(kT, J, B, newConfig)
    for i in range(nDataCol):
        newConfig = MCstepFunction(kT, J, B, newConfig)
        if i % sampleInterval == 0:
            Esamples.append(energyFunction(newConfig,J,B))
            Msamples.append(abs(netMagnetizationPerSpin(newConfig)))
    Esamples = np.array(Esamples)
    Msamples = np.array(Msamples)
    return [Esamples.mean()/len(config)**2, # Mean Energy per spin
           Esamples.var()*len(config)**2/(kT**2), # Magnet Suspectibility per spin
           Msamples.mean(), # Average net magnetization per spin
           np.var(Esamples)/(len(config)*kT**2), # Heat Capacity
           newConfig] # The Final configuration

### Functions for MCSteps

We have two main functions within the next Code Block: ```energyIsing2D(config,J,B)``` and ```MCstepPBC(kT,J,B,config)```. There are multiple things to note that with our calculations though: 
- Our Up and Down interactions are always favorable! So those interactions will always be $J=-1$! 
- Our left and right interactions are always alternating! What does that mean though? Take the following example:

Let our column decision be the first column, then let our interaction with its left partner be $J=-1$ (favorable interaction) and its right partner interaction be $J=1$(unfavorable interaction) . Now, go to the second column. The interactions for the second column must alternate from the first column, thus it's left partner interactions is $J=1$ (unfavorable interaction) and its its right partner interaction are $J=-1$ (favorable interaction). This behavior will alternate back and forth throughout the whole code, creating this type of setup for the $J$ interactions: 

```if (col % 2) = 0 -> J_left = -1, J_right =1``` 

and then, ```else -> J_left = 1, J_right =-1```

And, this will create a system where you have two columns of the same spin changing back and forth! Knowing this, we have changed our MCStep and energyIsing2D steps to take into account these interactions! 

In [ ]:
def energyIsing2D(config,J,B):
    """
    Function for calculating the energy of an Ising Model.
    Requires a config (the Ising Model in question), a value of J (interaction parameter)
    and a value for B (external field parameter)
    """
    energyB,energyJ = 0,0 # Initializes local variables
    for row in range(len(config)): # Loop over rows
        for col in range(len(config)): # Simultaneously loop over columns
            energyB += B*config[row,col]
            """Note: the interactions up and down are always favorable which implies J=-1"""
            if row>0: # Up
                J = -1 
                energyJ += J*config[row-1,col]*config[row,col]
            else: # Periodic boundary conditions
                J = -1 
                energyJ += J*config[len(config)-1,col]*config[row,col] 
            if row<len(config)-1: # Down
                J = -1 
                energyJ += J*config[row+1,col]*config[row,col]
            else: # Periodic boundary conditions
                J = -1 
                energyJ += J*config[0,col]*config[row,col] 
                
            """
            Alternating Interactions Behavior:
            Odd Columns: J_right=-1, J_left=1; 
            Even Columns: J_right=1, J_left=-1
            """
            if( col % 2 == 1): # Odd Columns
                J_right = -1
                J_left = 1
            else: # Even Columns
                J_right = 1
                J_left = -1

            if col>0: # Left
                energyJ += J_left*config[row,col-1]*config[row,col]
            else: # Periodic boundary conditions
                energyJ += J_left*config[row,len(config)-1]*config[row,col] 
            if col<len(config)-1: # right
                energyJ += J_right*config[row,col+1]*config[row,col]
            else: # Periodic boundary conditions
                energyJ += J_right*config[row,0]*config[row,col]

    return energyB + energyJ/2 # Removes double counting
def MCstepPBC(kT,J,B,config):
    """MC step for Periodic Ising Model"""
    newConfig = config.copy()
    row,col = np.random.randint(0,len(config),2)
    Estart = config[row,col]*B
    """Note: the interactions up and down are always favorable which implies J=-1"""
    if row>0: # Up
        J = -1 
        Estart += J*config[row-1,col]*config[row,col] 
    else: # Periodic boundary conditions
        J = -1 
        Estart += J*config[len(config)-1,col]*config[row,col] 
    if row<len(config)-1: # Down
        J = -1 
        Estart += J*config[row+1,col]*config[row,col] 
    else: # Periodic boundary conditions
        J = -1 
        Estart += J*config[0,col]*config[row,col]

    """
    Alternating Interactions Behavior:
    Odd Columns: J_right=-1, J_left=1; 
    Even Columns: J_right=1, J_left=-1
    """    
    if( col % 2 == 1): # Odd Columns 
        J_right = -1
        J_left = 1
    else: # Even Columns
        J_right = 1
        J_left = -1

    if col>0: # Left
        Estart += J_left*config[row,col-1]*config[row,col] 
    else: # Periodic boundary conditions
        Estart += J_left*config[row,len(config)-1]*config[row,col] 
    if col<len(config)-1: # right
        Estart += J_right*config[row,col+1]*config[row,col]
    else: # Periodic boundary conditions
        Estart += J_right*config[row,0]*config[row,col] 
    Eend = -Estart

    if Eend < Estart: #did energy decrease?
      newConfig[row,col] *= -1 # Then flip the spin
    else:
      if np.all(np.random.random() <= np.exp(-(Eend-Estart)/kT)): # If a random number is less than the probability
        newConfig[row,col] *= -1 # Then flip the spin

    return newConfig # Return the potentially altered array

#### Defining Various Configurations
I define the following configurations below to start our equilibrium: 
- ```Stripes```: a striped configuration where stripes swap every two columns
- ```checkboardConfig```: original checkboard configuration
- ```array_negative```: an configuration of all negative values
- ```array_positive```: an configuration of all positive values

In [ ]:
lattice_size = 10 # Size of our lattice

# Defining Stripes configuration below
Stripes = np.zeros((lattice_size, lattice_size))
# Fill alternating columns with -1 and 1
Stripes[:,::4] = -1 
Stripes[:,1::4] = -1  
Stripes[:,2::4] = 1   
Stripes[:,3::4] = 1   

# Original Checkboard Configuration
checkerboardConfig = np.array([[(-1)**(row+col) for row in range(lattice_size)] for col in range(lattice_size)])

array_negative = np.full((lattice_size, lattice_size), -1) # all negative configuration
array_positive = np.full((lattice_size, lattice_size), 1) # all positive configuration

# Running Simulations for Each Configuration
We will now run a simulations for each configuration described above. Here are the conditions we are running for the configurations
- Equilibration Steps: We are running 1,000,000 equilibration steps before we start collecting our data
- Data Collection: we calculate our data for 20,000 samples after 1,000,000 equilibration steps
- Sample Size: sample every 1000 steps (we have an array of 10 samples for calculations)
- $J$ is already setup within our simulation
- $B$ is zero
- Temperature Values: we run our simulation for 20 kT values between 1 to 4

We calculate the following values within our ```runMC``` step: 
- Mean Energy Spin: held within ```mean_energy_spin```
- Mean Net Magnetization: held within ```mc_spin``` 
- Heat Capacity: held within ```heat_capacity```
- Magnet Suspectibility: held within ```magnet_suspect```
- Final Configuration: held within ```configurations```

We plot every value (except for configurations) against temperature afterwards. 

### Magnetic Field, Self-Imposed Question

We also decided to run a thought experiement about the magnetic field ```B```. We wanted to see how the heat capacity is affected as we increase ```B```, so we ran two simulations with the following conditions: 
- ```B=0``` (first four configurations described above)
- ```B=1```
- ```B=2```
Note: we run this code with the ```array_positive``` configuration. We will show though that no matter the initial configuration, we run enough equilibration steps for the intitial configuration to not matter. 

### Simulation for CheckerBoard below

In [ ]:
kT_values = np.linspace(1, 4, 20) # kT_values for simulation
nEquil = 1000000 # One Million Equilibration Steps
data_collect = 20000 # Twenty Thousand data collection steps
sample_size = 1000 # Sample every 1000 steps
J=-1 # J is defined for fun
B=0  # No magnetic field for this question

mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []

for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, checkerboardConfig, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Explanation of Plots and Answering Questions
These are the plots of magnetic susceptibility, net magnetization, and mean energy, done with the ```checkerboardConfig``` as the initial configuration. The magnetic susceptibility graph is the most telling for the behavior of the system for different temperatures. The phase transition occurs at 2.3 kT, which is aligns with the phase transtion seen in 12-5. That system shows a phase transtion at around 2.3 kT. However, the key difference is that at 2.3 kT the 12-5 system has a mean magnetization of 0.7, however, this system has a mean magnetization of 0.07 at that value. This means that even though the distribution of the spins is extremely different the phase transition still occurs at the same value. This difference in behavior can be attributed to the interactions between the rows and columns. The antiferromagnetic and ferromagnetic changes between rows and consistency in columns and significant equilibration period allows for the system to reach a similar phase transition at b=0. Under no magnetic field.

#### Animation for Checkboard (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, checkerboardConfig)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani

### Simulation for Stripes below:

In [ ]:
mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []

for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, Stripes, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Animation for Stripes (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, Stripes)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani

### Simulation for ```array_negative``` below:

In [ ]:
mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []

for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, array_negative, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Animation for ```array_negative``` (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, array_negative)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani

### Simulation for ```array_positive``` below:

In [ ]:
mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []

for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, array_positive, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Animation for ```array_positive``` (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, array_positive)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani

## Simulation Results and Answering Hard Drive Question: 

We can see after running all four configurations that given 1,000,000 equilibration steps, we can the same results for our heat capacity, mean energy, and net magnetization! Thus, initial configuration does not matter for this question. 

---

We now wish to answer the question on why the difference in phase transititon is useful for measuring the magnetic field in a hard drive. Difference in phase transition, as show in the next two simulations where ```B=1``` and ```B=2```, will be created because our phase transition will shift to the left as the magnetic field increases! Thus, we can tell that a difference with phase transition will cause us to have a a proportionality relationship with the magnetic field! We can also understand that hard drives are striped in nature (we are simulating a hard drive), therefore we can understand/see how the magnetic field will affect the phase transition of a hard drive. It also can be assumed that the hard drive, when it occurs as a phase transition, will stop working because the temperature is too hard for a hard drive. Therefore, we can now understand how the magnetic field is connected with the difference in phase transition which also lets us understand how well our hard drive will work at various temperatures! 

---

Now, we can see below the shift in phase transition after those simulations run! 

### Simulation for ```B=1``` below:

In [ ]:
mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []
B=1 # Simluation at B=1
for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, array_positive, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Animation for ```B=1``` (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, array_positive)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani

### Simulation for ```B=2``` below:

In [ ]:
mean_energy_spin = []
mc_spin = []
magnet_suspect = []
heat_capacity = []
configurations = []
B=2 # Simluation at B=2
for Temp in kT_values:
    results = runMC(Temp, J, B, nEquil, data_collect, sample_size, array_positive, MCstepPBC, energyIsing2D)
    mean_energy_spin.append(results[0]) 
    magnet_suspect.append(results[1]) 
    mc_spin.append(results[2]) 
    heat_capacity.append(results[3])
    configurations.append(results[4])

#### Plotting following Graphs: 

- 'Mean Net Magnetization vs Temperature'
- 'Mean Energy vs Temperature'
- 'Magnet Suspectibility vs Temperature'
- 'Heat Capacity vs Temperature' (in separate block)

In [ ]:
# Plot the results using Seaborn
plt.figure(figsize=(12,8))

plt.subplot(2, 2, 1)
plt.plot(kT_values, mc_spin, marker='o')
plt.title('Mean Net Magnetization vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Net Magnetization')

plt.subplot(2, 2, 2)
plt.plot(kT_values, mean_energy_spin, marker='o')
plt.title('Mean Energy vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Mean Energy')

plt.subplot(2, 2, 3)
plt.plot(kT_values, magnet_suspect, marker='o')
plt.title('Magnet Suspectibility vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Magnet Suspectibility')


plt.tight_layout()
plt.show()

In [ ]:
plt.plot(kT_values, heat_capacity, marker='o')
plt.title('Heat Capacity vs Temperature')
plt.xlabel('Temperature (kT)')
plt.ylabel('Heat Capacity')

#### Animation for ```B=2``` (only 1500 steps)

In [ ]:
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
fig, ax = plt.subplots()
images = runMCmovie(1.5,-1,0, MCstepPBC, ax, array_positive)
ani = animation.ArtistAnimation(fig, images, interval=1000/50, blit=True,repeat_delay=500)

In [ ]:
ani